# From CV results to separated in CSVs

In [33]:
from os import walk
import os
import pandas as pd
import numpy as np
from itertools import product

In [37]:
folder = os.path.join(".", "results", "")
mse = "mean squared error"
pl = "percent labeled"

metric = 'accuracy score'

In [26]:
results_found = next(walk(folder), (None, None, []))[2]
sorted(results_found)

['base.csv',
 'r10.csv',
 'r15.csv',
 'r20.csv',
 'r25.csv',
 'r30.csv',
 'r35.csv',
 'r5.csv']

In [28]:
dfs = []
for index, r in enumerate(results_found):
    dfs.append(pd.read_csv(folder + results_found[index]))

df = pd.concat(dfs, ignore_index=True)
df.drop(["fold", "Unnamed: 0"], axis=1, inplace=True)


In [30]:
df.head(10)

,dataset,percent labeled,base,filter,f1-score,mean squared error,accuracy score
0,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.561039,2.272727,0.636364
1,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.521212,1.000000,0.545455
2,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.369697,4.909091,0.454545
3,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.484848,1.909091,0.454545
4,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.654545,1.272727,0.727273
5,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.561039,1.909091,0.636364
6,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.466667,3.000000,0.400000
7,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.460000,1.400000,0.500000
8,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.800000,0.500000,0.800000
9,BreastTissue.csv,0.3,KNeighborsClassifier,ENN,0.800000,2.000000,0.800000


In [66]:
filters = df['filter'].unique()
base_estimators = df['base'].unique()
percents_labeled = df[pl].unique()
datasets = df['dataset'].unique()
percents_labeled.sort()

print(f"Base estimators used: {base_estimators}")
print(f"Filters used: {filters}")
print(f"Percents labeled used: {percents_labeled}")
print(f"# Datasets used: {len(datasets)}")

Base estimators used: ['KNeighborsClassifier' 'DecisionTreeClassifier' 'GaussianNB']
Filters used: ['ENN' 'LSSm' 'ENANE' 'base']
Percents labeled used: [0.05 0.1  0.15 0.2  0.25 0.3  0.35]
# Datasets used: 18


In [67]:
percents_bases = np.array(list(product(percents_labeled, base_estimators)))
print("Pairs of base estimator with filters:\n", percents_bases)

Pairs of base estimator with filters:
 [['0.05' 'KNeighborsClassifier']
 ['0.05' 'DecisionTreeClassifier']
 ['0.05' 'GaussianNB']
 ['0.1' 'KNeighborsClassifier']
 ['0.1' 'DecisionTreeClassifier']
 ['0.1' 'GaussianNB']
 ['0.15' 'KNeighborsClassifier']
 ['0.15' 'DecisionTreeClassifier']
 ['0.15' 'GaussianNB']
 ['0.2' 'KNeighborsClassifier']
 ['0.2' 'DecisionTreeClassifier']
 ['0.2' 'GaussianNB']
 ['0.25' 'KNeighborsClassifier']
 ['0.25' 'DecisionTreeClassifier']
 ['0.25' 'GaussianNB']
 ['0.3' 'KNeighborsClassifier']
 ['0.3' 'DecisionTreeClassifier']
 ['0.3' 'GaussianNB']
 ['0.35' 'KNeighborsClassifier']
 ['0.35' 'DecisionTreeClassifier']
 ['0.35' 'GaussianNB']]


In [43]:
grouped_df = df.groupby(['dataset', 'percent labeled',
                                              'base', 'filter']).mean()

grouped_df = grouped_df[metric].to_frame()
grouped_df.reset_index(inplace=True)
grouped_df.shape

(1512, 5)

In [49]:
grouped_df.head(10)

,dataset,percent labeled,base,filter,accuracy score
0,BreastTissue.csv,0.05,DecisionTreeClassifier,ENANE,0.247273
1,BreastTissue.csv,0.05,DecisionTreeClassifier,ENN,0.262727
2,BreastTissue.csv,0.05,DecisionTreeClassifier,LSSm,0.260909
3,BreastTissue.csv,0.05,DecisionTreeClassifier,base,0.319091
4,BreastTissue.csv,0.05,GaussianNB,ENANE,0.225455
5,BreastTissue.csv,0.05,GaussianNB,ENN,0.197273
6,BreastTissue.csv,0.05,GaussianNB,LSSm,0.170909
7,BreastTissue.csv,0.05,GaussianNB,base,0.246364
8,BreastTissue.csv,0.05,KNeighborsClassifier,ENANE,0.330909
9,BreastTissue.csv,0.05,KNeighborsClassifier,ENN,0.200000


In [114]:
for percent, base in percents_bases:
    percent = float(percent)
    
    data = grouped_df.loc[(grouped_df['percent labeled'] == percent) & (grouped_df['base'] == base)]
    data['base:filter'] = data['base'] + ':' + data['filter']
    data.drop(columns=['percent labeled', 'base', 'filter'], inplace=True)
    
    regex = base+'.*'
    
    filtered_data = data[data['base:filter'].str.match(regex)]
    
    new_df = {'datasets': datasets}
    for dataset in datasets:
        new_data = data.loc[data['dataset'] == dataset]
        new_cols = new_data['base:filter']
        
        data_to_add = list()
        for index, row in new_data.iterrows():
            try:
                data_to_add = np.append(new_df[row['base:filter']], row[metric])
                new_df[row['base:filter']] = data_to_add
            except KeyError:
                new_df[row['base:filter']] = [row[metric]]
    
       
    new_df = pd.DataFrame(new_df)
    
    new_df.to_csv(os.path.join('alg_datasets', f'{percent}_{base}.csv'), index=False)
   


/var/folders/t7/v35czvpd0l74_n7b5l8k71kc0000gn/T/ipykernel_5179/747642043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['base:filter'] = data['base'] + ':' + data['filter']
